# Aplicação de Sistema de Recomendação (Notebook 2)
## Projeto da disciplina **SCC0284  - Sistemas de Recomendação**
### Sistemas de Recomendação aplicado no modelo de Draft da NBA.

## Membros

* Arthur Santorum Lorenzetto - 12559465
* Gustavo Silva de Oliveira - 12567231

##  Breve resumo:

Nesse notebook iremos realizar o calculo da recomendação de fato usando o método de recomendação baseada em conteúdo. Ou seja, iremos calcular as similaridades do nosso conjunto de treino com os nossos metadados, somar elas para cada componente do treino para obter nosso score, ordenar do maior para o menor afim de receber as 15 primeiras recomendações e, por fim, calcular a precisão usando o Average Precision.

In [120]:
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

In [121]:
# lendo os dados que obtemos no notebook anterior.
df_metadados = pd.read_csv('./advanced_stats_2009_2020.csv')
df_treinamento = pd.read_csv('./advanced_stats_2021.csv')
lottery_players = pd.read_csv('./lottery_drafted_players.csv')

# usando o arquivo lottery players para refinar nossos metadados, visto que ele possui todos os jogadores do college de 2009 até
# 2020, e estamos interessados apenas naqueles que foram TOP15 em seus respectivos drafts.
lista_jogadores = lottery_players['player_name'].unique().tolist()
df_metadados_lottery = df_metadados.query('player_name in @lista_jogadores')
df_metadados_lottery

In [122]:
# o arquivo jogadores contem todos os jogadores do college que se alistaram para o draft de 2021, iremos usar ele para refinar
# nosso banco de dados de treinamento, visto que ele possui TODOS os jogadores de college de 2021 e não é nisso que estamos
# interessados.

# dados retirados de: https://247sports.com/Article/2021-NBA-Draft-underclassmen-List-of-college-players-declaring-early-162966399/

jogadores = pd.read_csv('jogadores.csv')
jogadores = jogadores['Jogadores']
df_treinamento = df_treinamento.query("player_name in @jogadores")
df_treinamento

,Unnamed: 0,player_name,player_name_id,GP (<10),GP (>=10 e < 20),GP (>=20 e < 26),GP (>=26 e < 30),GP (>=30),Min_per (<4.3),Min_per (>=4.3 e < 22.04),...,blk_per (<0.0),blk_per (>=0.0 e < 0.5),blk_per (>=0.5 e < 1.3),blk_per (>=1.3 e < 2.9),blk_per (>=2.9),stl_per (<0.7),stl_per (>=0.7 e < 1.3),stl_per (>=1.3 e < 1.75),stl_per (>=1.75 e < 2.4),stl_per (>=2.4)
15,15,AJ Lawson,15,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
19,19,Aamir Simms,19,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
28,28,Aaron Henry,28,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
33,33,Aaron Wiggins,33,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
107,107,Alan Griffin,107,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4676,4676,Ty Gadsden,4676,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4748,4748,Tyson Etienne,4748,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4801,4801,Warith Alatishe,4801,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
4879,4879,Yves Pons,4879,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0


nos três próximos blocos calculamos as similaridades entre os metadados e o treinamento utilizando cosseno de forma semelhante ao que fizemos em um dos exercícios. Optamos por deixar comentado pela demora de execução, sendo assim, executamos uma fez, coletamos as similaridades e convertemos para csv de forma que possamos otimizar nosso trabalho.

In [123]:
# tam_metadados = len(df_metadados_lottery['player_name_id'])
# tam_treinamento = len(df_treinamento['player_name_id'])

# ids_treinamento = df_treinamento['player_name_id'].tolist()
# ids_metadados_lottery = df_metadados_lottery['player_name_id'].tolist()

In [124]:
# similarity = pd.DataFrame(columns=['player_test_id','player_dados_id','similarity'])

# for i in ids_treinamento:
#     x1 = df_treinamento.loc[df_treinamento['player_name_id'] ==i].copy()
#     x1_novo = x1.drop(columns = ['Unnamed: 0','player_name', 'player_name_id']).values
    
#     for j in ids_metadados_lottery:
#         y1 = df_metadados_lottery.loc[df_metadados_lottery['player_name_id'] == j].copy()
#         y1_novo = y1.drop(columns = ['Unnamed: 0','player_name', 'player_name_id']).values    
    
#         value = cosine_similarity(x1_novo , y1_novo)

#         line = {'player_test_id':i, 'player_dados_id':j, 'similarity':value}
#         similarity = similarity.append(line, ignore_index=True)

#     print(i)
# similarity = similarity.astype({'similarity':'float'})
# similarity

In [125]:
# similarity.to_csv("similarity.csv")
# similarity

In [126]:
similarity = pd.read_csv("similarity.csv")
similarity

,Unnamed: 0,player_test_id,player_dados_id,similarity
0,0,15,144,0.400000
1,1,15,180,0.133333
2,2,15,373,0.133333
3,3,15,510,0.466667
4,4,15,548,0.400000
...,...,...,...,...
30655,30655,4927,23420,0.133333
30656,30656,4927,23504,0.133333
30657,30657,4927,23646,0.200000
30658,30658,4927,23705,0.066667


In [127]:
# calculando o score para cada atleta do treinamento e tomando os 15 primeiros como recomendação (pegando apenas o id deles)

scores = similarity.groupby(by=["player_test_id"]).sum().reset_index()
scores = scores.sort_values(by=['similarity'], ascending = False)
recomendacao = scores.head(15)
recomendacao = recomendacao[['player_test_id']]
recomendacao

,player_test_id
68,1613
142,3179
94,2262
162,3494
90,2164
23,705
98,2341
161,3472
170,3706
1,19


In [128]:
# através do id, agora iremos pegar o nome das 15 primeiras recomendações do nosso modelo

players_list = []
x = 0

for index, row in recomendacao.iterrows():
    x = row['player_test_id']
    player = df_treinamento.loc[df_treinamento['player_name_id'] == x]
    player_name = player['player_name'].values
    players_list.append(player_name)

players_list
new_list = []
for x in players_list:
    new_list.append(x[0])
    
new_list

['Evan Mobley',
 'Luka Garza',
 'Jared Butler',
 'Micah Potter',
 'Jalen Suggs',
 'Cameron Krutwig',
 'Jay Huff',
 'McKinley Wright IV',
 'Neemias Queta',
 'Aamir Simms',
 'Joe Wieskamp',
 'Terry Taylor',
 'Joel Ayayi',
 'Corey Kispert',
 'Keve Aluma']

In [129]:
# para testar nossa recomendação, tomamos os jogadores do college que REALMENTE foram selecionados nas 15 primeiras 
# posições do draft de 2021 e iremos calcular o Average Precision.

# dados retirados de: https://www.nba.com/news/2021-nba-draft-results-picks-1-60


draft_verdadeiro_college = ['Cade Cunningham','Evan Mobley','Scottie Barnes','Jalen Suggs', 'Franz Wagner', 'Davion Mitchell',
                           'Ziaire Williams','James Bouknight','Joshua Primo','Chris Duarte','Moses Moody','Corey Kispert',
                           'Trey Murphy III','Tre Mann','Kai Jones']

In [130]:
# calculando o Average Precision

def AveragePrecision(lista1, lista2):
    total_relevantes_recomendados = 0
    relevantes_recomendados = []
    for x in lista2:
        if(x in lista1):
            relevantes_recomendados.append(x)
            total_relevantes_recomendados = total_relevantes_recomendados + 1
    
    if(total_relevantes_recomendados == 0):
        return -1
    soma = 0
    value = 1
    for x in relevantes_recomendados:
        index = lista2.index(x)+1
        soma = soma + (value/index)
        value = value+1
    
    valor_AP = soma / total_relevantes_recomendados
    return valor_AP

 
valor_AP = AveragePrecision(draft_verdadeiro_college, new_list)
print(valor_AP)

0.5380952380952381


Podemos observar que obtemos um Average Precision razoável, isso se explica pois, por mais que tenhamos acertado poucos jogadores (3 jogadores da nossa recomendação realmente foram selecionados no top15), esses que acertamos estavam bem próximos da sua posição verdadeira de seleção, e o Average Precision recompensa isso.